In [1]:
!pip install scikit-learn

In [2]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

path1="/content/drive/MyDrive/ML TATA power/MLTATAPOWER_TRAIN.csv"
path2="/content/drive/MyDrive/ML TATA power/ML EVAL.csv"
# Loading dataset.
train_df = pd.read_csv(path1) # training data variable
predict_df = pd.read_csv(path2) # testing data variable


train_df.columns = train_df.columns.str.replace(' ', '_')
predict_df.columns = predict_df.columns.str.replace(' ', '_')

train = train_df.copy();
test = predict_df.copy();

In [4]:

# Defining features and target variable for training
X_train = train_df[['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY']]
y_train = train_df['HEALTHINESS']

# Standardizing features for training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Building the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Defining features for prediction
X_predict = predict_df[['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY']]

# Standardizing features for prediction
X_predict = scaler.transform(X_predict)

# Making predictions on the prediction dataset
predicted_probabilities = model.predict(X_predict)

i=2

# Printing the predicted probabilities for the prediction dataset
print("\nPredicted Healthiness percentage for ",i," index of the Evaluating dataset : ",predicted_probabilities[i]*100,"%")


Epoch 1/10
634/634 [==============================] - 3s 3ms/step - loss: 0.3503 - accuracy: 0.8310 - val_loss: 0.2382 - val_accuracy: 0.8939
Epoch 2/10
634/634 [==============================] - 2s 3ms/step - loss: 0.2254 - accuracy: 0.8999 - val_loss: 0.2039 - val_accuracy: 0.9132
Epoch 3/10
634/634 [==============================] - 2s 4ms/step - loss: 0.1921 - accuracy: 0.9178 - val_loss: 0.1761 - val_accuracy: 0.9237
Epoch 4/10
634/634 [==============================] - 3s 4ms/step - loss: 0.1662 - accuracy: 0.9299 - val_loss: 0.1572 - val_accuracy: 0.9347
Epoch 5/10
634/634 [==============================] - 2s 4ms/step - loss: 0.1447 - accuracy: 0.9411 - val_loss: 0.1378 - val_accuracy: 0.9446
Epoch 6/10
634/634 [==============================] - 3s 5ms/step - loss: 0.1270 - accuracy: 0.9496 - val_loss: 0.1238 - val_accuracy: 0.9521
Epoch 7/10
634/634 [==============================] - 5s 7ms/step - loss: 0.1148 - accuracy: 0.9558 - val_loss: 0.1045 - val_accuracy: 0.9540
Epoch 

In [5]:
train.head()

,FLU_GAS_TEMP,STEAM_PRESSURE,TURBINE_SPEED,GENERATOR_VOLTAGE,AIR_QUALITY,HEALTHINESS,ERROR_TYPE
0,598,2950,2896,16770,359,0,2
1,349,2303,3620,18296,194,0,1
2,799,2080,2980,16691,134,0,1
3,515,2485,3553,17586,213,1,0
4,529,2090,3024,17354,116,1,0


In [6]:
#Error description
error_desc=['NO ERROR','ERROR A','ERROR B','ERROR C']

# Defining features and target variable for training (for ERROR_TYPE)
X_train_error = train_df[['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY']]
y_train_error = train_df['ERROR_TYPE'].astype(int)  # Ensure labels are integers

# Standardizing features for training (for ERROR_TYPE)
scaler_error = StandardScaler()
X_train_error = scaler_error.fit_transform(X_train_error)

# Converting NumPy array back to DataFrame
X_train_error = pd.DataFrame(X_train_error, columns=['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY'])

# Splitting the dataset into training and validation sets (for ERROR_TYPE)
X_train_error, X_val_error, y_train_error, y_val_error = train_test_split(
    X_train_error, y_train_error, test_size=0.2, random_state=42
)

# Building the DNNClassifier for ERROR_TYPE
feature_columns_error = [tf.feature_column.numeric_column(key=key) for key in X_train_error.columns]
classifier_error = tf.estimator.DNNClassifier(
    feature_columns=feature_columns_error,
    hidden_units=[128, 64],
    n_classes=4,  # Assuming 4 classes for ERROR_TYPE
)

# Input function for ERROR_TYPE
input_fn_error = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_train_error, y=y_train_error, batch_size=32, num_epochs=10, shuffle=True
)

# Training the DNNClassifier for ERROR_TYPE
classifier_error.train(input_fn=input_fn_error)

# Defining features for prediction (for ERROR_TYPE)
X_predict_error = predict_df[['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY']]

# Standardizing features for prediction (for ERROR_TYPE)
X_predict_error = scaler_error.transform(X_predict_error)

# Converting NumPy array back to DataFrame
X_predict_error = pd.DataFrame(X_predict_error, columns=['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY'])

# Input function for prediction (for ERROR_TYPE)
predict_input_fn_error = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_predict_error, shuffle=False
)

# Making predictions on the prediction dataset (for ERROR_TYPE)
predicted_classes_error = list(classifier_error.predict(input_fn=predict_input_fn_error))

# Extracting predicted classes
predicted_classes_error = [prediction['class_ids'][0] for prediction in predicted_classes_error]

# Printing the predicted classes for the prediction dataset (for ERROR_TYPE)
print("The error type is : ")
print(error_desc[predicted_classes_error[0]])

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instruc

The error type is : 
NO ERROR


In [7]:
# Error description
error_desc = ['NO ERROR', 'ERROR A', 'ERROR B', 'ERROR C']

# Defining features and target variable for training (for ERROR_TYPE)
X_train_error = train_df[['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY']]
y_train_error = train_df['ERROR_TYPE'].astype(int)  # Ensure labels are integers

# Standardizing features for training (for ERROR_TYPE)
scaler_error = StandardScaler()
X_train_error = scaler_error.fit_transform(X_train_error)

# Splitting the dataset into training and validation sets (for ERROR_TYPE)
X_train_error, X_val_error, y_train_error, y_val_error = train_test_split(
    X_train_error, y_train_error, test_size=0.2, random_state=42
)

# Building the DNN model for ERROR_TYPE using tf.keras
model_error = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_error.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')  # Assuming 4 classes for ERROR_TYPE
])

# Compile the model
model_error.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_error.fit(X_train_error, y_train_error, epochs=10, batch_size=32, validation_data=(X_val_error, y_val_error))

# Defining features for prediction (for ERROR_TYPE)
X_predict_error = predict_df[['FLU_GAS_TEMP', 'STEAM_PRESSURE', 'TURBINE_SPEED', 'GENERATOR_VOLTAGE', 'AIR_QUALITY']]

# Standardizing features for prediction (for ERROR_TYPE)
X_predict_error = scaler_error.transform(X_predict_error)

# Making predictions on the prediction dataset (for ERROR_TYPE)
predicted_probabilities_error = model_error.predict(X_predict_error)

# Extracting predicted classes
predicted_classes_error = np.argmax(predicted_probabilities_error, axis=1)

# Printing the predicted classes for the prediction dataset (for ERROR_TYPE)
print("\nThe error type for index ",i," of the evaluating dataset is: ",error_desc[predicted_classes_error[i]])

Epoch 1/10
634/634 [==============================] - 3s 3ms/step - loss: 0.5351 - accuracy: 0.7897 - val_loss: 0.2604 - val_accuracy: 0.9045
Epoch 2/10
634/634 [==============================] - 2s 3ms/step - loss: 0.2167 - accuracy: 0.9269 - val_loss: 0.1719 - val_accuracy: 0.9458
Epoch 3/10
634/634 [==============================] - 1s 2ms/step - loss: 0.1675 - accuracy: 0.9445 - val_loss: 0.1394 - val_accuracy: 0.9529
Epoch 4/10
634/634 [==============================] - 1s 2ms/step - loss: 0.1436 - accuracy: 0.9493 - val_loss: 0.1270 - val_accuracy: 0.9509
Epoch 5/10
634/634 [==============================] - 1s 2ms/step - loss: 0.1270 - accuracy: 0.9552 - val_loss: 0.1102 - val_accuracy: 0.9607
Epoch 6/10
634/634 [==============================] - 1s 2ms/step - loss: 0.1184 - accuracy: 0.9577 - val_loss: 0.1098 - val_accuracy: 0.9582
Epoch 7/10
634/634 [==============================] - 1s 2ms/step - loss: 0.1080 - accuracy: 0.9593 - val_loss: 0.0948 - val_accuracy: 0.9667
Epoch 